In [1]:
from cobra import *
import numpy as np
import re
from cobra.flux_analysis import *

In [2]:
# model = io.mat.load_matlab_model('../humanModel.mat')  # read model for MATLAB file

In [3]:
__author__ = "Nantia Leonidou"
__description__ = " Detect inactive reactions using either fastFVA oder FASTCC "

from cobra.flux_analysis import flux_variability_analysis, fastcc
from cobra.flux_analysis import find_blocked_reactions
from cobra import *
from cobra.io.sbml import *


####
# import optlang
# optlang.available_solvers  --> to see which solvers are available
####

def find_inactive_rxns(model, method=1):
    """ ** Runnable only in jupyter notebook **
        Find all inactive (blocked) reactions in a cobra model using either FASTCC or fastFVA methods
        Input: model and method: fastFVA(1, default) or FASTCC (2)
        Output: list of inactive reactions """

    # inactive_rxns = []
    if method == 1:  # fastFVA (by default, if no method number is given)

        print("Checking all reactions (fastFVA)...")
        # set all objective coefficients to 0
        for react in range(len(model.reactions)):
            model.reactions[react].objective_coefficient = 0

        # inactive_rxns = find_blocked_reactions(model)
        ####################################################################
        #      **       ALTERNATIVELY:          **
        # following code delivers the same inactive reactions
        ###################################################################
        # conduct FVA analysis
        fva = flux_variability_analysis(model)
        # store all maximum and all minimum flux-values after FVA
        opt_max = []
        opt_min = []
        for i in range(len(fva)):
            opt_max.append(fva.iloc[i][1])
            opt_min.append(fva.iloc[i][0])
            
        # set constraints to consider a reaction "inactive"
        abs_max = [i for i in range(len(opt_max)) if abs(opt_max[i]) < 1e-6]
        abs_min = [i for i in range(len(opt_min)) if abs(opt_min[i]) < 1e-6]

        is_inactive = [rxn_id for rxn_id in abs_max if rxn_id in abs_min]
        # get all (inactive) blocked reactions' id
        inactive_rxns = []
        for rxn_id in is_inactive:
            inactive_rxns.append(model.reactions[rxn_id].id)
        #########################################################################

    else:  

        print("Checking all reactions (FASTCC)...")
        # get non-blocked reactions, * in MATLAB the fastcc function gives less is_active reactions *
        # Moreover: if fastcc(model) runs several times, it would have different number of non-blocked reactions
        # which makes sense based on the present state of the implementation. Because, flux solutions are usually not unique,
        # so there may be multiple solutions giving the same maximum.
        is_active = fastcc(model, zero_cutoff=1e-4).reactions
        # get all active reactions' id
        active_rxns = [rxn.id for rxn in is_active]
        # get all model's reactions
        all_model_rxns = [rxns.id for rxns in model.reactions]
        # get difference, which are then the inactive reactions
        inactive_rxns = list(set(all_model_rxns) - set(active_rxns))
        # ** returns ~1307 inactive reactions, less than the MATLAB code **

    print('Model consists of %s blocked (inactive) reactions' % len(inactive_rxns))
    return inactive_rxns

In [4]:
lengths = []
rxns = []
# run fastCC for 20 times to see different outputs (number of inactive reactions)
for i in range(20):
    model = io.read_sbml_model('../RECON1.xml')
    fast_cc = find_inactive_rxns(model, 2)
    lengths.append(len(fast_cc))
    rxns.append(fast_cc)

Checking all reactions (FASTCC)...
Model consists of 1302 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model consists of 1319 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model consists of 1304 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model consists of 1316 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model consists of 1297 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model consists of 1311 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model consists of 1302 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model consists of 1305 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model consists of 1299 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model consists of 1302 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model consists of 1294 blocked (inactive) reactions
Checking all reactions (FASTCC)...
Model co

In [ ]:
# MATLAB gives 1274 inactive reactions 

In [47]:
with open('../matlab_fastcc_inactiveRxns.txt', 'r') as file:
    matlab_inactive_rxns = [line.strip('\n').strip("''") for line in file]
print(len(matlab_inactive_rxns))

1274


In [48]:
matlab_inactive_rxns

['11DOCRTSLtm',
 '11DOCRTSLtr',
 '11DOCRTSTRNtm',
 '11DOCRTSTRNtr',
 '1a_24_25VITD2Hm',
 '1a_24_25VITD3Hm',
 '1a_25VITD2Hm',
 '1a_25VITD3Hm',
 '25VITD2Hm',
 '25VITD3Hm',
 '34DHPHAMT',
 '34HPLFM',
 '3DPHBH1',
 '3DPHBH2',
 '3HBCDm',
 '3HKYNAKGAT',
 '3HPCOAHYD',
 '3HPPD',
 '3HXKYNDCL',
 '3HXKYNOXDA',
 '3M4HDXPAC',
 '3MOXTYROX',
 '3NTD7l',
 '42A12BOOX',
 '4HBZCOAFm',
 '4HBZFm',
 '5HOXINDACTO2OX',
 '5HOXINDACTOX',
 '5HOXINDACTOXm',
 '5HOXINOXDA',
 '5HTRPDOX',
 '5HXKYNDCL',
 '5HXKYNOXDA',
 '6HTSTSTERONEte',
 '6HTSTSTERONEtr',
 'A4GALTc',
 'A4GALTg',
 'A4GNT1g',
 'A4GNT2g',
 'ABO1g',
 'ABO2g',
 'ABO3g',
 'ABO4g',
 'ABO5g',
 'ABO6g',
 'ABO7g',
 'ABO8g',
 'ABO9g',
 'ABUTt2rL',
 'ACACT1x',
 'ACACT4p',
 'ACACT5p',
 'ACACT6p',
 'ACACT7p',
 'ACACT8p',
 'ACACT9p',
 'ACCOAgt',
 'ACCOAtr',
 'ACGAGBSIDEtg',
 'ACGAGBSIDEtl',
 'ACGALFUCGALACGALFUC12GAL14ACGLCGALGLUSIDEte',
 'ACGALFUCGALACGALFUC12GAL14ACGLCGALGLUSIDEtg',
 'ACGALFUCGALACGALFUCGALACGLCGAL14ACGLCGALGLUSIDEte',
 'ACGALFUCGALACGALFUCGALACGLCGA

In [51]:
differences_rxns = []
for idx,r in enumerate(rxns):
    print('List Nr:%s'%(idx))
    differences_rxns.append(list(set(r) - set(matlab_inactive_rxns)))
    print(list(set(r) - set(matlab_inactive_rxns)))

List Nr:0
['EX_6dhf_e', 'EX_25hvitd3_e', 'EX_udp_e', 'INSTt2', 'SRTNtu', 'EX_inost_e', 'HISTAtu', 'NNATr_copy2', 'GCHOLAte', 'CRNt_copy1', 'EX_5thf_e', 'EX_5dhf_e', 'EX_s2l2n2m2masn_e', 'FACOAL240', 'EX_ncam_e', 'EX_glygn4_e', 'LDH_D', 'EX_lys__L_e', 'EX_lgnc_e', 'LEUtec', 'RAI4', 'GLYtp', 'EX_6thf_e', 'EX_10fthf5glu_e', 'EX_arachd_e', 'STCOAtx', 'EX_10fthf6glu_e', 'RDH2']
List Nr:1
['EX_so4_e', 'EX_6dhf_e', 'RDH2a', 'EX_25hvitd3_e', 'EX_udp_e', 'PCHOL_HSter', 'TCHOLAt', 'SRTNtu', 'PVD3', 'NDPK5', 'ARACHDt2', 'EX_glygn5_e', 'HISTAtu', 'EX_chsterol_e', 'FATP2t', 'GCHOLAte', '7DHCHSTEROLtr', 'DHAAt1r', 'RIBt', 'CRNt_copy1', 'EX_5thf_e', 'EX_strch2_e', 'EX_5dhf_e', 'ADKd', 'EX_s2l2n2m2masn_e', 'EX_mag_hs_e', 'DRIBt', 'FATP9t', 'EX_ncam_e', 'VD3', 'ILEtec', 'EX_11_cis_retfa_e', 'EX_spc_hs_e', 'EX_ump_e', 'NDPK8m', 'SPC_HSt', 'EX_lys__L_e', 'EX_dag_hs_e', 'PROSTGE2t2', 'LEUtec', 'EX_drib_e', 'EX_6thf_e', 'EX_10fthf5glu_e', 'STCOAtx', 'EX_10fthf6glu_e']
List Nr:2
['EX_25hvitd3_e', 'PCHOL_HSt